In [12]:
# 대한민국 헌법을 PDF파일로 입력받아,
# RAG 모델을 만들어 Ollama로 대통령의 탄핵 요건에 대한 질문과 답변을 생성 (250601)

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

In [24]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100) 

In [25]:
# 디렉토리 내 모든 파일을 리스트로 변환하는 함수 정의

import os

def list_files(directory):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_list.append(os.path.join(root, file))
    return file_list

# 지정된 디렉토리 내 모든 파일명을 리스트로 호출
file_names = list_files('./data')
print(file_names)

['./data\\constitution_of_Korea.pdf']


In [26]:
from langchain.embeddings import HuggingFaceEmbeddings

In [27]:
# 버전 문제 해결 : https://chan7ee.tistory.com/entry/issue-tensorflow-keras-transformer-%EB%B2%84%EC%A0%84-%EB%AC%B8%EC%A0%9C
# pip install tf-keras==2.16.0

# 문장을 임베딩으로 변환하고 벡터 저장소에 저장
embeddings_model = HuggingFaceEmbeddings(
    model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',    # 다국어 모델
    # model_name='jhgan/ko-sroberta-multitask',  # 한국어 모델 - 에러 발생 (250603)
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

embeddings_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, multi_process=False, show_progress=False)

In [28]:
loader = PyMuPDFLoader(file_names[0])
documents = loader.load()

split_doc = loader.load_and_split(text_splitter)

# 임베딩 DB 생성
db = Chroma.from_documents(
    documents=split_doc, embedding=embeddings_model, collection_name="my_db"
)

In [ ]:
# 파일로 DB 저장하기


In [29]:
# 유사성 측정

question = "대통령"
searchDocs = db.similarity_search(question)

print(searchDocs[0].page_content)
print(searchDocs[1].page_content)
print(searchDocs[3].page_content)

민족문화의 창달에 노력하여 대통령으로서의 직책을 성실히 수행할 것을 국민 앞에 엄숙히 선
서합니다."
 
제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
 
제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정
한 국무위원의 순서로 그 권한을 대행한다.
 
제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책
을 국민투표에 붙일 수 있다.
 
제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강
화를 한다.
민족문화의 창달에 노력하여 대통령으로서의 직책을 성실히 수행할 것을 국민 앞에 엄숙히 선
서합니다."
 
제70조 대통령의 임기는 5년으로 하며, 중임할 수 없다.
 
제71조 대통령이 궐위되거나 사고로 인하여 직무를 수행할 수 없을 때에는 국무총리, 법률이 정
한 국무위원의 순서로 그 권한을 대행한다.
 
제72조 대통령은 필요하다고 인정할 때에는 외교ㆍ국방ㆍ통일 기타 국가안위에 관한 중요정책
을 국민투표에 붙일 수 있다.
 
제73조 대통령은 조약을 체결ㆍ비준하고, 외교사절을 신임ㆍ접수 또는 파견하며, 선전포고와 강
화를 한다.
법제처                                                            15                                                       국가법령정보센터
「대한민국헌법」
제96조 행정각부의 설치ㆍ조직과 직무범위는 법률로 정한다.
 
                    제4관 감사원
 
제97조 국가의 세입ㆍ세출의 결산, 국가 및 법률이 정한 단체의 회계검사와 행정기관 및 공무원
의 직무에 관한 감찰을 하기 위하여 대통령 소속하에 감사원을 둔다.
 
제98조 ①감사원은 원장을 포함한 5인 이상 11인 이하의 감사위원으로 구성한다.
②원장은 국회의 동의를 얻어 대통령이 임명하고, 그 임기는 4년으로 하며, 1차에 한하여 중임
할 수

In [30]:
retriever = db.as_retriever(search_kwargs={"k": 4})

In [31]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama

# Prompt 템플릿 생성
template = '''Answer the question based only on the following context:
{context} Please answer all the answers in Korean.:

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

model = ChatOllama(model = "llama3")  

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
# query = "대통령이 탄핵될 수 있는 조건에 대해 알려줘"
query = "대통령의 의무에 대해 알려주세요"
answer = rag_chain.invoke(query)

print("Query:", query)
print("Answer:", answer)

Query: 대통령의 의무에 대해 알려주세요
Answer: 대통령의 의무는 다음과 같습니다.

제90조 : 국가원로자문회의를 둘 수 있고, 조직ㆍ직무범위를 정할 수 있습니다.
제91조 : 국가안전보장회의를 둘 수 있고, 조직ㆍ직무범위를 정할 수 있습니다.
제92조 : 민주평화통일자문회의를 둘 수 있고, 조직ㆍ직무범위를 정할 수 있습니다.
제93조 : 국민경제자문회의를 둘 수 있고, 조직ㆍ직무범위를 정할 수 있습니다.

따라서 대통령의 의무는 국가원로자문회의, 국가안전보장회의, 민주평화통일자문회의, 국민경제자문회에 대한 조직ㆍ직무범위의 정함입니다.
